<a href="https://colab.research.google.com/github/RosemaryAustin/AMIL_Projects/blob/master/Rosemary_Austin_T04_06_%5B00%5D_Using_Pre_Trained_Models_Project_%5BColab%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Copyright 2019 Google LLC.

In [0]:
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Using Pre-Trained Models Project

In this project we will import a pre-existing model that recognizes objects and use the model to identify those objects in a video. We'll edit the video to draw boxes around the identified object and then reassemble the video so that the boxes are shown around objects in the video.

## Overview

### Learning Objectives

* Use OpenCV to process images and video.
* Use a pre-trained model to identify and label objects in each frame of a video.
* Make judgements about classification quality and when to apply predicted labels.

### Prerequisites

* Classification
* Saving and Loading Models
* OpenCV
* Video Processing

### Estimated Duration

330 minutes (285 minutes working time, 45 minutes for presentations)

### Deliverables

1. A copy of this Colab notebook containing your code and responses to the ethical considerations below. The code should produce a functional labeled video.
1. A group presentation. After everyone is done, we will ask each group to stand in front of the class and give a brief presentation about what they have done in this lab. The presentation can be a code walkthrough, a group discussion, a slide show, or any other means that conveys what you did over the course of the day and what you learned. If you do create any artifacts for your presentation, please share them in the class folder.

### Grading Criteria

This project is graded in separate sections that each contribute a percentage of the total score:

1. Building and Using a Model (80%)
1. Ethical Implications (10%)
1. Project Presentation (10%)

#### Building and Using a Model

There are 6 demonstrations of competency listed in the problem statement below. Each competency is graded on a 3 point scale for a total of 18 available points. The following rubric will be used:

| Points | Description |
|--------|-------------|
| 0      | No attempt at the competency |
| 1      | Attempted competency, but in an incorrect manner |
| 2      | Attempted competency correctly, but sub-optimally |
| 3      | Successful demonstration of competency |


#### Ethical Implications

There are six questions in the **Ethical Implications** secion. Each question is worth 2 points. The rubric for calculating those points is:

| Points | Description |
|--------|-------------|
| 0      | No attempt at question or answer was off-topic or didn't make sense |
| 1      | Question was answered, but answer missed important considerations  |
| 2      | Answer adequately considered ethical implications |

#### Project Presentation

The project presentation will be graded on participation. All members of a team should actively participate.

## Team

Please enter your team members names in the placeholders in this text area:

*   Prashamsa 
*   Christine
*   Rosemary



# Exercises

## Exercise 1: Coding

For this workshop you will process a video frame-by-frame, identify objects in each frame, and draw a bounding box and label around each object.
 
Use the [SSD MobileNet V1 Coco](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md) 'ssd_mobilenet_v1_coco' model. The video that you'll be processing can be [found on Pixabay](https://pixabay.com/videos/cars-motorway-speed-motion-traffic-1900/). The 640x360 version of the video is smallest and easiest to handle, though any should work since you must scale down the images for processing.
 
**Graded** demonstrations of competency:
1. Obtain the pre-trained model from the [TensorFlow Zoo](https://github.com/tensorflow/models).
1. Load the pre-trained model into a TensorFlow object.
1. Obtain a video file from Pixabay to use for classification.
1. Process the video frame-by-frame creating a modified output video.
1. Apply a classification model to an image.
1. Draw a bounding box around classified objects in each image.
 
The flow of the program is roughly:
 
* Read in a video file (use the one in this colab if you want)
* Load the TensorFlow model
* Loop over each frame of the video
> * Scale the frame down to a size the model expects
 * Feed the frame to the model
 * Loop over detections made by the model
 >  * If the detection score is above some threshold draw a bounding box onto the frame and put a label in or near the box
   * Write the frame back to a new video
 
Some tips:
 
* Processing an entire video is slow, consider truncating the video or skipping over frames during development. Skipping frames will make the video choppy, but you'll be able to see a wider variety of images than you would in a truncated video with all of the original frames in the clip.
* The model expects a 300x300 image. You'll likely have to scale your frames to fit the model. When you get a bounding box that box is relative to the scaled image. You'll need to scale the bounding box out to the original image size.
* Don't start by trying to process the video. Instead, capture one frame and work with it until you are happy with your object detection, bounding boxes, and labels. Once you get those done worry about video handling.
* The [Coco labels file](https://github.com/nightrome/cocostuff/blob/master/labels.txt) can be used to identify classified objects.
 

### Student Solution

In [0]:
#This is loading in the video file

import cv2 as cv
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd

cars_video = cv.VideoCapture('Cars.mp4')

height = int(cars_video.get(cv.CAP_PROP_FRAME_HEIGHT))
width = int(cars_video.get(cv.CAP_PROP_FRAME_WIDTH))
fps = cars_video.get(cv.CAP_PROP_FPS)
total_frames = int(cars_video.get(cv.CAP_PROP_FRAME_COUNT))



In [0]:
#This is uploading the model 

import urllib.request
import os

base_url = 'http://download.tensorflow.org/models/object_detection/'
file_name = 'ssd_mobilenet_v1_coco_2018_01_28.tar.gz'

url = base_url + file_name

urllib.request.urlretrieve(url, file_name)

os.listdir()

In [0]:
# Extract the Model Data
import tarfile
import shutil

dir_name = file_name[0:-len('.tar.gz')]

if os.path.exists(dir_name):
  shutil.rmtree(dir_name) 

tarfile.open(file_name, 'r:gz').extractall('./')

os.listdir(dir_name)

In [0]:
#The detections that the model returns
outputs = (
    'num_detections',
    'detection_classes',
    'detection_scores',
    'detection_boxes',
)

In [0]:
#creating a dictionary with just id and lables 
#receiving the id and the display_name from the data
with open('mscoco_complete_label_map.pbtxt.txt', 'r') as file:
    data = file.read().replace('\n', '')
labels = {}
for val in data.split('{')[1:]:
  label = val.split('display_name: ')[1].split('}')[0]
  ident = val.split('id: ')[1].split(' ')[0]
  labels[ident] = label

In [0]:
labels

In [0]:
# looking at the detected lables and storing that in a list
def label_dec (image_f, lable1):
  labels_ = []
  for var in image_f[1][0]:
    if var!=1.0:
      labels_.append(labels[str(int(var))[:2]])
  return (labels_)

In [0]:
# brings the model into a graph

import tarfile
import shutil
import tensorflow as tf
import matplotlib.pyplot as plt

dir_name = file_name[0:-len('.tar.gz')]

if os.path.exists(dir_name):
  shutil.rmtree(dir_name) 

tarfile.open(file_name, 'r:gz').extractall('./')

os.listdir(dir_name)

In [0]:
#This puts the model in a graph so that it can be used in tensorflow
frozen_graph = os.path.join(dir_name, 'frozen_inference_graph.pb')

with tf.gfile.FastGFile(frozen_graph,'rb') as f:
  graph_def = tf.GraphDef()
  graph_def.ParseFromString(f.read())

In [0]:
#This recolors the frame and resizes it in readable form
def frame_fix(image):
  frame = cv.cvtColor(image, cv.COLOR_BGR2RGB)
  frame = cv.resize(frame, (300, 300))
  return frame


In [0]:
#This is bringing in a diffrent label file 
#as the original one was returning the wrong detections
#creating a dictionary with just id and lables 
#receiving the id and the display_name from the data
open_new = open("labels (1).txt", "r")
label2={}
for x in open_new:
  key,_,value=x.partition(':')
  value,_,_=value.partition('\n')
  label2[key]=value

# label1

In [0]:
#detections[1][0]

In [0]:
#This the amount of frames in the Cars.mp4

total_frames = int(cars_video.get(cv.CAP_PROP_FRAME_COUNT))

#This is makeing a place holder for the new labled video to be stored in
fourcc = cv.VideoWriter_fourcc(*'mp4v')
output_video = cv.VideoWriter('Cars-short.mp4', fourcc, fps, (300, 300))


# cycles through the frames of the Cars.mp4 at a ten step
for current_frame in range(0, total_frames,10): 
  cars_video.set(cv.CAP_PROP_POS_FRAMES, current_frame)
  ret, frame = cars_video.read()
  #if the program crashes shows us where it failed in the loop
  if not ret:
    raise Exception(f'Problem reading frame {current_frame} from video')
  
  #This passes the frame to be formated
  frame = frame_fix(frame)
  image = [frame] #image into a list 
  
  #starts tensor flow sess to create the graph 
  with tf.Session() as sess:
    sess.graph.as_default()
    tf.import_graph_def(graph_def, name='')
    
    #passes it to the model to find objects in frame
    detections = sess.run(
        [sess.graph.get_tensor_by_name(f'{op}:0') for op in outputs],
        feed_dict={ 'image_tensor:0': image })
    
    # Must be diffrent names other wise it will over ride it ever loop
    label1 = label_dec(detections, label2)

    
    #making data frame to hold the results of model outputs
    df1 = pd.DataFrame()
    top = detections[3][0][:len(label1)][:,0]
    left = detections[3][0][:len(label1)][:,1]
    bottom = detections[3][0][:len(label1)][:,2]
    right = detections[3][0][:len(label1)][:,3]
    scale = 1.0

    df1['labels'] = label1
    df1['top'] = top
    df1['left'] = left
    df1['bottom'] = bottom
    df1['right'] = right
    
    #print(df1)
    
    #This draws the boxes around the objects detected
    width,height,colour = frame.shape
    for index, row in df1.iterrows():
      left = int(row['left']*height)
      top = int(row['top']*width)
      right = int(row['right']*height)
      bottom = int(row['bottom']*width)
      #if row['labels'] == '"cars"':
      frame = cv.rectangle(frame,(left,top),(right,bottom),[0,255,0],2)
      frame = cv.putText(frame, df1['labels'][index] , (left,top), cv.FONT_HERSHEY_SIMPLEX,scale,[0,255,0],2)
    output_video.write(frame)
#     plt.imshow(frame)
#     plt.show()
  
  
  
    

In [0]:
#releases the videos from the RAM
cars_video.release()
output_video.release()

## Exercise 2: Ethical Implications

Even the most basic of models have the potential to affect segments of the population in different ways. It is important to consider how your model might positively and negative effect different types of users.

In this section of the project you will reflect on the positive and negative implications of your model.

Frame the context of your models creation using this narriative:\n",

  > The city of Seattle is attempting to reduce traffic congestion in their downtown area. As part of this project, they plan on allowing each local driver one free downtown trip per week. After that the driver will have to pay a $50 toll for each extra day per week driven. As an early proof-of-concept for this project your team is tasked with using machine learning to correctly identify automobiles on the road. The next phase of the project will involve detecting license plate numbers and then cross-referencing that data with RFID chips that should be mounted in all local drivers cars."


### Student Solution

**Positive Impact**

Your model is trying to solve a problem. Think about who will benefit from that problem being solved and write a brief narrative about how the model will help.

---

The local police force has been tasked with discovering the cause for so many accidents on heavly travled road. Thanks to the model they are able to indetify that people and animals are crossing the road. They were then able to put up a human and animal crossing sign cutting down on the number of accidents happening on that road.

**Negative Impact**

Models don't often have universal benefit. Think about who might be negatively impacted by the predictions your model is making. This person or persons might not be directly using the model, but instead might be impacted indirectly.

---

The local police force has started using the model. However the model is misidentifying objects on the road. Things that should be appearing as cars are appearing as humans and animals instead. The crossing signs they have put up  have not resolved the cause for so many accidents and wasted limited tax money.

**Bias**

Models can be bias for many reasons. The bias can come from the data used to build the model (eg. sampling, data collection methods, available sources) and from the interpretation of the predictions generated by the model.

Think of at least two ways that bias might have been introduced to your model and explain both below.

---

One source of bias in the model could be the model training data, where the classifcation of the objects are limited.

Another source of bias in the model could be the image quality that the cameras are feeding into the model to idetify.

**Changing the Dataset to Mitigate Bias**

Bias datasets are one of the primary ways in which bias is introduced to a machine learning model. Look back at the input data that you fed to your model. Think about how you might change something about the data to reduce bias in your model.

What change or changes could you make to your dataset less bias? Consider the data that you have, how and where that data was collected, and what other sources of data might be used to reduce bias.

Write a summary of change that could be made to your input data.

---

Since we are looking at cars, a better dataset that has more detail description about car, car types, models, vechile types, street signs could be used that fits the purpose of the model. 

The second option will be to use a dataset that can classifiy more objects.

**Changing the Model to Mitigate Bias**

Is there any way to reduce bias by changing the model itself? This could include modifying algorithmic choices, tweaking hyperparameters, etc.

Write a brief summary of changes that you could make to help reduce bias in your model.

---

*   Since the model has potential bias of limited classification of model training data, it could be improved to expand the classification of objects in the traning data to mitigate bias..

*   Since the model has potential bias of image quality, it could be improved to feed higher quality images to the model to mitigate bias.

**Mitigating Bias Downstream**

Models make predictions. Downstream processes make decisions. What processes and/or rules should be in place for people and systems interpreting and acting on the results of your model to reduce the bias? Describe these below.

---

Beside the prediction from the model, the downstream processes could add the rules of verifying other reasons that might relate to the  traffic safety. They can be the assurance of traffic lights and traffic signs in good condition and function well. Also, they can be the estimation of weather impact including wind, rain or snow.